In [ ]:
import pickle
from matplotlib import pyplot as plt
import numpy as np
import json
from sklearn.metrics.pairwise import euclidean_distances
import cv2

In [ ]:
matlab_points = pickle.load(open('/root/data/alok/biomass_estimation/playground/image_to_checkerboard_points_matlab.pkl', 'rb'))
circular_points = pickle.load(open('/root/data/alok/biomass_estimation/playground/image_to_checkerboard_points_circular.pkl', 'rb'))

In [ ]:
matlab_left = matlab_points['/data/alok/biomass_estimation/playground/validation_images/left/left_frame.png']['left_corners']
matlab_right = matlab_points['/data/alok/biomass_estimation/playground/validation_images/left/left_frame.png']['right_corners']
circular_left = circular_points['/data/alok/biomass_estimation/playground/validation_images/left/left_frame.png']['left_corners']
circular_right = circular_points['/data/alok/biomass_estimation/playground/validation_images/left/left_frame.png']['right_corners']

In [ ]:
(matlab_left - matlab_right) - (circular_left - circular_right)

In [ ]:
matlab_wcm = pickle.load(open('/root/data/alok/biomass_estimation/playground/wcm_matlab.pkl', 'rb'))
circular_wcm = pickle.load(open('/root/data/alok/biomass_estimation/playground/wcm_circular.pkl', 'rb'))

In [ ]:
diffs = np.squeeze(1e3 * (euclidean_distances(world_coordinate_matrix_matlab) - euclidean_distances(world_coordinate_matrix_circular))).flatten()

In [ ]:
ds = []
for i in range(world_coordinate_matrix_matlab.shape[0]):
    for j in range(i+1, world_coordinate_matrix_matlab.shape[0]):
        d = np.linalg.norm(world_coordinate_matrix_matlab[i, :] - world_coordinate_matrix_matlab[j, :])
        ds.append(d)
        
ds = np.array(ds)
diffs = 1e3 * ds - matlab_df.ground_truth_distance_mms.values
plt.hist(diffs)
plt.show()

In [ ]:
circular_params

In [ ]:
ds = []
for i in range(world_coordinate_matrix_circular.shape[0]):
    for j in range(i+1, world_coordinate_matrix_circular.shape[0]):
        d = np.linalg.norm(world_coordinate_matrix_circular[i, :] - world_coordinate_matrix_circular[j, :])
        ds.append(d)
        
ds = np.array(ds)
diffs = 1e3 * ds - circular_df.ground_truth_distance_mms.values
plt.hist(diffs)
plt.show()

In [ ]:
diffs = np.squeeze(1e3 * (euclidean_distances(matlab_wcm[0]) - euclidean_distances(circular_wcm[0]))).flatten()

In [ ]:
plt.hist(diffs)
plt.show()

In [ ]:
matlab_params = json.load(open('/root/data/alok/biomass_estimation/playground/2020-01-14T00_00_00Z_L40013180_R40029775_stereo-parameters.json'))
circular_params = json.load(open('/root/data/alok/biomass_estimation/playground/EstimatedStereoCamParams_L40013180_R40029775.json'))

In [ ]:
matlab_params['TranslationOfCamera2']

In [ ]:
circular_params['TranslationOfCamera2']

In [ ]:
matlab_params['CameraParameters1']['FocalLength']

In [ ]:
circular_params['CameraParameters1']['FocalLength']

In [ ]:
(matlab_params['CameraParameters1']['FocalLength'][0] * matlab_params['TranslationOfCamera2'][0]) / 608

In [ ]:
circular_params['CameraParameters1']['FocalLength'][0] * circular_params['TranslationOfCamera2'][0] / 600

In [ ]:
matlab_df = pickle.load(open('/root/data/alok/biomass_estimation/playground/analysis_df_matlab.pkl', 'rb'))
circular_df = pickle.load(open('/root/data/alok/biomass_estimation/playground/analysis_df_circular.pkl', 'rb'))

In [ ]:
plt.hist(matlab_df[matlab_df.ground_truth_distance_mms > 400].predicted_distance_mms - circular_df[matlab_df.ground_truth_distance_mms > 400].predicted_distance_mms)
plt.show()

In [ ]:
plt.hist(matlab_df.predicted_distance_mms - matlab_df.ground_truth_distance_mms)
plt.show()

In [ ]:
plt.hist(circular_df.predicted_distance_mms - circular_df.ground_truth_distance_mms)
plt.show()

In [ ]:
((0.605)**3 - (0.60)**3) / (0.60)**3

In [ ]:
circular_cameraMatrix1 = np.array(circular_params['CameraParameters1']['IntrinsicMatrix']).transpose() 
circular_cameraMatrix2 = np.array(circular_params['CameraParameters2']['IntrinsicMatrix']).transpose()

In [ ]:
matlab_cameraMatrix1 = np.array(matlab_params['CameraParameters1']['IntrinsicMatrix']).transpose() 
matlab_cameraMatrix2 = np.array(matlab_params['CameraParameters2']['IntrinsicMatrix']).transpose()
R = np.array(matlab_params['RotationOfCamera2']).transpose()
T = np.array(matlab_params['TranslationOfCamera2']).transpose()
world_checkerboard_points_list = []
for left_corners, right_corners in zip(matlab_left, matlab_right):
    P1 = np.dot(matlab_cameraMatrix1, np.hstack([np.eye(3), np.array([[0], [0], [0]])]))
    P2 = np.dot(matlab_cameraMatrix2, np.hstack([R, T[:, np.newaxis]]))
    wcp = cv2.triangulatePoints(P1, P2, left_corners, right_corners).T
    world_checkerboard_points = wcp[:,:3] / wcp[:,3, None]
    world_checkerboard_points_list.append(world_checkerboard_points)
    
world_coordinate_matrix_matlab = np.empty([len(world_checkerboard_points_list), 3])
for i, world_checkerboard_points in enumerate(world_checkerboard_points_list):
        world_coordinate_matrix_matlab[i, :] = world_checkerboard_points

world_coordinate_matrix_matlab = np.array([world_coordinate_matrix_matlab[:, 0],
                                           world_coordinate_matrix_matlab[:, 2],
                                           -world_coordinate_matrix_matlab[:, 1]]).T * 1e-3

In [ ]:
circular_cameraMatrix1 = np.array(circular_params['CameraParameters1']['IntrinsicMatrix']).transpose() 
circular_cameraMatrix2 = np.array(circular_params['CameraParameters2']['IntrinsicMatrix']).transpose()
R = np.array(circular_params['RotationOfCamera2']).transpose()
T = np.array(circular_params['TranslationOfCamera2']).transpose()
world_checkerboard_points_list = []
for left_corners, right_corners in zip(circular_left, circular_right):
    P1 = np.dot(circular_cameraMatrix1, np.hstack([np.eye(3), np.array([[0], [0], [0]])]))
    P2 = np.dot(circular_cameraMatrix2, np.hstack([R, T[:, np.newaxis]]))
    wcp = cv2.triangulatePoints(P1, P2, left_corners, right_corners).T
    world_checkerboard_points = wcp[:,:3] / wcp[:,3, None]
    world_checkerboard_points_list.append(world_checkerboard_points)
    
world_coordinate_matrix_circular = np.empty([len(world_checkerboard_points_list), 3])
for i, world_checkerboard_points in enumerate(world_checkerboard_points_list):
        world_coordinate_matrix_circular[i, :] = world_checkerboard_points

world_coordinate_matrix_circular = np.array([world_coordinate_matrix_circular[:, 0],
                                             world_coordinate_matrix_circular[:, 2],
                                             -world_coordinate_matrix_circular[:, 1]]).T * 1e-3

In [ ]:
(world_coordinate_matrix_circular - circular_wcm) * 1e3